# Embedding Sentence

In [42]:
import torch
from transformers import AutoModel, AutoTokenizer
from underthesea import word_tokenize
phobert = AutoModel.from_pretrained("vinai/phobert-base-v2")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")


sentence = 'Hệ thống thoát nước là hệ thống dẫn hay vận chuyển nước đi từ chỗ mực nước cao đến nơi có mực nước thấp hơn.'

sentence=word_tokenize(sentence, format="text")
input_ids = torch.tensor([tokenizer.encode(sentence)])

with torch.no_grad():
    features = phobert(input_ids)

with torch.no_grad():
    outputs = phobert(input_ids)
    hidden_states = outputs.last_hidden_state
sentence_embedding = hidden_states.mean(dim=1).squeeze()
embedding_vector = sentence_embedding.numpy()

print("Kích thước embedding:", embedding_vector.shape)
print( embedding_vector)

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Kích thước embedding: (768,)
[ 4.65169959e-02  7.33712018e-02 -9.05355066e-02  2.72403389e-01
 -3.59702110e-02  1.05468720e-01 -5.52208759e-02  1.47267282e-02
  5.81741221e-02  1.42505780e-01  1.63550302e-02  1.99643075e-01
 -5.05824685e-02 -1.23532504e-01 -1.82169408e-01 -1.62908316e-01
  1.58492252e-01  1.27213374e-01  1.63962856e-01  8.98863077e-02
 -6.58350214e-02 -2.54924688e-02  2.98370302e-01  2.51838297e-01
 -3.64151627e-01 -1.48102762e-02  1.07239023e-01  1.09780235e-02
  2.54030190e-02 -7.68124089e-02 -5.78849465e-02  6.87036365e-02
 -4.85526323e-02  3.44654955e-02  4.75755900e-01  4.65383649e-01
 -1.36399552e-01  1.84333399e-02 -3.49493064e-02  2.95189261e-01
  4.07338478e-02  1.80351317e-01 -4.03652228e-02 -5.30847944e-02
  1.21131493e-02 -8.16733390e-02  1.03905704e-02 -9.41843092e-02
  2.33353134e-02 -6.70577362e-02  2.19662860e-01  5.15189990e-02
 -2.83985934e-03  2.41691247e-01  6.53617233e-02 -3.53125781e-02
  1.97910368e-01 -4.12913978e-01  9.15753394e-02  1.12378918e